# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [60]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [61]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"




@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [62]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [4]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [52]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

21188774108950298909796

## Use this to import default tbears wallet and run the transactions


In [14]:
password = "test1_Account"
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
print(deployer_wallet.get_address())
icon_service.get_balance(deployer_wallet.get_address())

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb


786640639999000000000000000

In [63]:
contracts_file_custom = "shubh_yeouido_contracts_20210112084238.pkl"

In [64]:
with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

# Deploying all contracts


In [35]:

# contracts = {}
deploy = ['addressProvider', 'daoFund','delegation','feeProvider','governance', 'lendingPool', 'lendingPoolCore','lendingPoolDataProvider','liquidationManager','ommToken','oToken', 'priceOracle','rewards', 'sample_token','snapshot','worker_token']
deploy = ['sicx']
# for directory in {"score"}:
# deploy= ['rewards', 'sample_token','snapshot','worker_token']
directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "sample_token":
       params = {'_name': "BridgeDollars",'_symbol':'USDb' ,'_decimals': 18}
    elif item == "omm_token":
        params = {'_initialSupply':0, '_decimals': 18}
    elif item == "worker_token":
        params = {'_initialSupply':100, '_decimals': 18}
    elif item == "sicx":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(res)
    contracts[item] = res.get('scoreAddress', '')

    

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xe5f3abcd02b2457a27e7156718051c9bad4fcbeacd23bb48839267220fe29329', 'blockHeight': 5020935, 'blockHash': '0x445bd7f1677dff03108ee374843c1ff3ca566b8f2032a4f8d5b06f3790ff2a11', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx75b4cd08dd95e1e6a8ed948653ccb413c5d0573f', 'stepUsed': 1053603200, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1053603200, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [65]:
pprint(contracts)

{'addressProvider': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37',
 'delegation': 'cx5ecc21ade5e4a90586bde6451da3b0d733dd11d6',
 'feeProvider': 'cx52555a5dda72b98a3ed692b58761f68af9486073',
 'governance': 'cx9e0ecfd9cc9a0c1cee8fc4a4701e970554835f54',
 'lendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'lendingPoolCore': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'lendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582',
 'liquidationManager': 'cx001794d12ea9170f4bb7a620e0a03a157f35a429',
 'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'oToken': 'cx77d34d57f7ea300637058ef65babc52da91376a4',
 'oUSDb': 'cx1aef9c66e508cfc32282e27e645ace726e831bd3',
 'omm_token': 'cxff036606b11066a142262431b089e8d0af145bef',
 'priceOracle': 'cx0b57375df6a814ca3f589e91b15015f135fbd046',
 'rewards': 'cxcccee742d4c1b7a3a84a20a41691d302e25ab3a0',
 'sample_token': 'cx9da823297393ecb193030b983ce334f25af20aac',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'snapshot': 

## deploying otoken for usdb

In [86]:
#deploy oUSDb
params = {"_name":"OmmUSDbInterestToken","_symbol":"oUSDb"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oUSDb'] = res.get('scoreAddress', '')


contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xf71114df51fda978919804b55e486b86b9b83aed18721c0100f2f1cfbf5f501d', 'blockHeight': 13205473, 'blockHash': '0x3f5a25f0ea22550cc3ed510752788101bccc9bb25c7a43f9397e277535cd2398', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx66788ee4dca134edae9ea9b8cc98c4e8073e644b', 'stepUsed': 1146141440, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1146141440, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'addressProvider': 'cx797bc63e7b27758816910cb35f0fa1f546b57c33',
 'daoFund': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
 'delegation': 'cx24e32367425660a67c91d6e3c1077914df4e0286',
 'feeProvider': 'cxd55f27870c9b219272ea4fc31d5d7ec8dc78a97e',
 'governance': 'cx672132f49f0ef02969cc6b223613206e46ff34b1',
 'lendingPool': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'lendingPoolCore': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
 'lendingPoolDataProvider': 'cx88ae036091243916d87cbc3b11f39e317dcd46ad',
 'liquidationManager': 'cx21b6ee09cd144b2a778bdcf69a46ef2cf898ff1c',
 'ommToken': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348',
 'oToken': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
 'priceOracle': 'cxe76fd817fae3b485ae6e4fea2cf88d050d379b63',
 'rewards': 'cx3e9b334a82529797818ca824cc58c51b52a26087',
 'sample_token': 'cxc88e39248bf428d3ab0b9c9fbafef9ff423c3a75',
 'snapshot': 'cx584f740fedab472057bd8ee6e32d97b9b0a05b9d',
 'worker_token': 'cxd4e8a5dd852439ccdec73a2b4cd973148f09775d',


## Deploying otoken for icx

In [32]:
#deploy oICX 
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x527bde8bcb020fe33fcca493322b5779bcf9efdab346b62ad82fecf8dfc2c2a6', 'blockHeight': 5020903, 'blockHash': '0x2261c8619d6b379953771487904e2e655ed366e001f11f1e877aaf9b64c9c30c', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx2bb91424c10a6ded8b5da01994c33b2b3cc99090', 'stepUsed': 1146047640, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1146047640, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

{'addressProvider': 'cx827be16fe4efbf2516ef80795301579c823288a6',
 'daoFund': 'cx97d4f10f4d793c35886b31cc8c70a872ea3b1448',
 'delegation': 'cx7eb823b737f355c08f1911009bb834ffb6773253',
 'feeProvider': 'cx09be6d8080af4be3e03a48a6c1b54955e1feefae',
 'governance': 'cxc3403b1adc439baa0e3553abb25cc6f290432d26',
 'lendingPool': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b',
 'lendingPoolCore': 'cx3755cee5c580d18c3c46b1816465e765a24d1b85',
 'lendingPoolDataProvider': 'cx52fd89a5beeb3a10747a4fd8981b7b866b9bb289',
 'liquidationManager': 'cx0242d3eb23c86a8bc32bf13f5681afc75371bec9',
 'ommToken': 'cxe2fe821536890b983bc16bdc6217f740c7e40f8b',
 'oToken': 'cxcee03a41d9b57fdf08359a9d40029796a47f100e',
 'priceOracle': 'cxd902d339f59a0c7344a8d9b92659d8bcaa9c70f9',
 'rewards': 'cx4f194dfd4db4bdc658be46985a9583a8808cf38e',
 'sample_token': 'cx195a1294e7836f3f865147e4752a7c7852362c43',
 'snapshot': 'cx1fe500bdb2d04ae8f0aa6e55e87a283b1e5b8f43',
 'worker_token': 'cx7f73b8beeda55f1ca97ebbdc5593b7d15c35376f',


## Deploying otoken for iusdc


In [33]:
#deploy oIUSDC
params = {"_name":"IconUSDInterest","_symbol":"oIUSDC","_decimals":6}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oIUSDC'] = res.get('scoreAddress', '')

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x10cb5bd2c9e772252feb345e614bce94962c99c75cc5164028175f11f33079dd', 'blockHeight': 5020909, 'blockHash': '0xff542c39c64e52738eee0c65d776e2a512b7ea3271d4a599ef99f252a521deb4', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxb75c665e8f0e257b9cbacddc6c7db713362cea99', 'stepUsed': 1146051760, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1146051760, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

{'addressProvider': 'cx827be16fe4efbf2516ef80795301579c823288a6',
 'daoFund': 'cx97d4f10f4d793c35886b31cc8c70a872ea3b1448',
 'delegation': 'cx7eb823b737f355c08f1911009bb834ffb6773253',
 'feeProvider': 'cx09be6d8080af4be3e03a48a6c1b54955e1feefae',
 'governance': 'cxc3403b1adc439baa0e3553abb25cc6f290432d26',
 'lendingPool': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b',
 'lendingPoolCore': 'cx3755cee5c580d18c3c46b1816465e765a24d1b85',
 'lendingPoolDataProvider': 'cx52fd89a5beeb3a10747a4fd8981b7b866b9bb289',
 'liquidationManager': 'cx0242d3eb23c86a8bc32bf13f5681afc75371bec9',
 'ommToken': 'cxe2fe821536890b983bc16bdc6217f740c7e40f8b',
 'oToken': 'cxcee03a41d9b57fdf08359a9d40029796a47f100e',
 'priceOracle': 'cxd902d339f59a0c7344a8d9b92659d8bcaa9c70f9',
 'rewards': 'cx4f194dfd4db4bdc658be46985a9583a8808cf38e',
 'sample_token': 'cx195a1294e7836f3f865147e4752a7c7852362c43',
 'snapshot': 'cx1fe500bdb2d04ae8f0aa6e55e87a283b1e5b8f43',
 'worker_token': 'cx7f73b8beeda55f1ca97ebbdc5593b7d15c35376f',


## Other Contract Addresses

In [89]:
contracts['sicx']="cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281"
contracts['staking']="cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c"
contracts['iusdc']="cx65f639254090820361da483df233f6d0e69af9b7"
contracts['usdb']="cxaa068556df80f9917ef146e889f0b2c4b13ab634"


In [34]:
pprint(contracts)

{'addressProvider': 'cx827be16fe4efbf2516ef80795301579c823288a6',
 'daoFund': 'cx97d4f10f4d793c35886b31cc8c70a872ea3b1448',
 'delegation': 'cx7eb823b737f355c08f1911009bb834ffb6773253',
 'feeProvider': 'cx09be6d8080af4be3e03a48a6c1b54955e1feefae',
 'governance': 'cxc3403b1adc439baa0e3553abb25cc6f290432d26',
 'lendingPool': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b',
 'lendingPoolCore': 'cx3755cee5c580d18c3c46b1816465e765a24d1b85',
 'lendingPoolDataProvider': 'cx52fd89a5beeb3a10747a4fd8981b7b866b9bb289',
 'liquidationManager': 'cx0242d3eb23c86a8bc32bf13f5681afc75371bec9',
 'oICX': 'cx2bb91424c10a6ded8b5da01994c33b2b3cc99090',
 'oIUSDC': 'cxb75c665e8f0e257b9cbacddc6c7db713362cea99',
 'oToken': 'cxcee03a41d9b57fdf08359a9d40029796a47f100e',
 'ommToken': 'cxe2fe821536890b983bc16bdc6217f740c7e40f8b',
 'priceOracle': 'cxd902d339f59a0c7344a8d9b92659d8bcaa9c70f9',
 'rewards': 'cx4f194dfd4db4bdc658be46985a9583a8808cf38e',
 'sample_token': 'cx195a1294e7836f3f865147e4752a7c7852362c43',
 'snapshot

## Saving the pkl file


In [58]:
now = datetime.utcnow()
filename =network +f' contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename = "kailash_custom.pkl"
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:

    pkl.dump(contracts, f)

saving contracts list as kailash_custom.pkl


## Updating contracts

In [38]:
contract_ = 'lendingPool'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

update lendingPool cx9ebdfb5b354db434de96b476c49a8fd455d96b1b


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xf5236e2a69816d175e86043b220bb2004e2d0acfdcbd624d054be4beb2229705',
 'blockHeight': 5021013,
 'blockHash': '0x6129e7cb212b86e8e30f56f602121b52fb65f31e8df2798b079043a6c43649e0',
 'txIndex': 1,
 'to': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b',
 'scoreAddress': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b',
 'stepUsed': 1739037600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1739037600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

# Configuring SCOREs's parameters

In [39]:
settings_lendingPool = [{'contract': 'lendingPool', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
                        {'contract': 'lendingPool', 'method': 'setLendingPoolDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
                        {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
                        {'contract': 'lendingPool', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
#                         {'contract': 'lendingPool', 'method': 'setStaking', 'params':{'_address': contracts['staking']}},
                        {'contract': 'lendingPool', 'method': 'setOICX', 'params':{'_address': contracts['oICX']}},
                        {'contract': 'lendingPool', 'method': 'setReward','params': {'_address':contracts['rewards']}},
                        {'contract': 'lendingPool', 'method': 'setSICX','params': {'_address':contracts['sicx']}},
                       ]
for sett in settings_lendingPool:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))



Calling setLendingPoolCore, with parameters {'_address': 'cx3755cee5c580d18c3c46b1816465e765a24d1b85'} on the lendingPool contract.
{'txHash': '0x9b3c0d76e5f09a50c5aa86cb1f523f18feb18674f1d16071812015cc8d47e8ac', 'blockHeight': 5021015, 'blockHash': '0x1fef1792b6ab2361e7ea5890a1c2201d3f5ffb0871d22dfc9e9e2aaa4b047361', 'txIndex': 1, 'to': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b', 'stepUsed': 151320, 'stepPrice': 10000000000, 'cumulativeStepUsed': 151320, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xc28ddb22f378b598e1cab5be008382d801ad9160b7245fad78e3cc3b029f671c', 'blockHeight': 5021016, 'blockHash': '0x0fe5748f30ae11dea0fe8815e8468e4fe6dd3179c7954c2ca10e60cd13f307d7', 'txIndex': 1, 'to': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b', 'stepUsed': 152920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 152920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xe4e942a3b0696ac9a6b0e3cb026c44a500efb2651b764b9acbb761eb51f5ed8d', 'blockHeight': 5021018, 'blockHash': '0x848e0de0db82863fb2dfb46cafa769923040a7c97d592da980fadcf7c04da8a9', 'txIndex': 1, 'to': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b', 'stepUsed': 151920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 151920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xbd829bce4a3ffc746c3e1c0a0d4e5e51a2d72f2d240184c98ec14cdf3e8c4b79', 'blockHeight': 5021020, 'blockHash': '0x359a176e67f25bc7048b2f0181e67c80f67ecb210ac0df6659a9b21e910ec53b', 'txIndex': 1, 'to': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b', 'stepUsed': 149120, 'stepPrice': 10000000000, 'cumulativeStepUsed': 149120, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x534c7d91fddd1822995388ef0068e36076e4faa9caa568289aa450e41e2225ed', 'blockHeight': 5021022, 'blockHash': '0xb8219b47946f6cf67357a5f4cd31afb13002967a8d9b7e29a8f436d80fc0da19', 'txIndex': 1, 'to': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b', 'stepUsed': 149120, 'stepPrice': 10000000000, 'cumulativeStepUsed': 149120, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [97]:
settings_lendinPoolDataProvider=[{'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['sample_token'],'_sym':"USDb"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"IUSDC"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setPriceOracle', 'params':{'_address': contracts['priceOracle']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPool', 'params':{'_address': contracts['lendingPool']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationManager', 'params':{'_address': contracts['liquidationManager']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setStaking', 'params':{'_address': contracts['staking']}}
            ]
for sett in settings_lendinPoolDataProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setSymbol, with parameters {'_reserve': 'cxc88e39248bf428d3ab0b9c9fbafef9ff423c3a75', '_sym': 'USDb'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setSymbol, with parameters {'_reserve': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281', '_sym': 'ICX'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setSymbol, with pa

In [40]:
settings_feeprovider =[{'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}}]
for sett in settings_feeprovider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLoanOriginationFeePercentage, with parameters {'_percentage': 1000000000000000} on the feeProvider contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


In [41]:
settings_oToken =[{'contract': 'oToken', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oToken', 'method': 'setReserve', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'oToken', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oToken', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}}]
for sett in settings_oToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx3755cee5c580d18c3c46b1816465e765a24d1b85'} on the oToken contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setReserve, with parameters {'_address': 'cx195a1294e7836f3f865147e4752a7c7852362c43'} on the oToken contract.
1


Calling setLendingPoolDataProvider, with parameters {'_address': 'cx52fd89a5beeb3a10747a4fd8981b7b866b9bb289'} on the oToken contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setLendingPool, with parameters {'_address': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b'} on the oToken contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


In [42]:
setting_priceOracle =[{'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
                      {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':10*10**17}},
                      {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'IUSDC','_quote':'IUSDC','_rate':10*10**17}}

                      ]
for sett in setting_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling set_reference_data, with parameters {'_base': 'USDb', '_quote': 'USD', '_rate': 1000000000000000000} on the priceOracle contract.
1


Calling set_reference_data, with parameters {'_base': 'ICX', '_quote': 'USD', '_rate': 1000000000000000000} on the priceOracle contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling set_reference_data, with parameters {'_base': 'IUSDC', '_quote': 'IUSDC', '_rate': 1000000000000000000} on the priceOracle contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


In [46]:
setting_addressProvider =[  {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']}},
                            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']}},
                            {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
                            {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
#                             {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}},
#                             {'contract': 'addressProvider', 'method': 'setIUSDC', 'params':{'_address':contracts['iusdc']}},
                            {'contract': 'addressProvider', 'method': 'setoIUSDC', 'params':{'_address':contracts['oIUSDC']}}]
for sett in setting_addressProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPool, with parameters {'_address': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b'} on the addressProvider contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setLendingPoolDataProvider, with parameters {'_address': 'cx52fd89a5beeb3a10747a4fd8981b7b866b9bb289'} on the addressProvider contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setUSDb, with parameters {'_address': 'cx195a1294e7836f3f865147e4752a7c7852362c43'} on the addressProvider contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setoUSDb, with parameters {'_address': 'cxcee03a41d9b57fdf08359a9d40029796a47f100e'} on the addressProvider contract.
1


Calling setsICX, with parameters {'_address': 'cx75b4cd08dd95e1e6a8ed948653ccb413c5d0573f'} on the addressProvider contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setoICX, with parameters {'_address': 'cx2bb91424c10a6ded8b5da01994c33b2b3cc99090'} on the addressProvider contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setoIUSDC, with parameters {'_address': 'cxb75c665e8f0e257b9cbacddc6c7db713362cea99'} on the addressProvider contract.
1


In [48]:
settings_reserves = [{'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['sample_token'],
                                                   "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"4{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}},
                     {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['sicx'],
                                                    "optimalUtilizationRate":f"6{'0'*17}",
                                                    "baseBorrowRate":f"0{'0'*17}",
                                                    "slopeRate1":f"8{'0'*16}",
                                                    "slopeRate2":f"2{'0'*18}"} ]}},
#                     {'contract': 'lendingPoolCore', 
#                         'method': 'setReserveConstants', 
#                         'params' :{"_constants": [{"reserve":contracts['iusdc'],
#                                                     "optimalUtilizationRate":f"8{'0'*17}",
#                                                    "baseBorrowRate":f"2{'0'*16}",
#                                                    "slopeRate1":f"4{'0'*16}",
#                                                    "slopeRate2":f"1{'0'*18}"} ]}}
                    ]
for sett in settings_reserves:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cx195a1294e7836f3f865147e4752a7c7852362c43', 'optimalUtilizationRate': '800000000000000000', 'baseBorrowRate': '20000000000000000', 'slopeRate1': '40000000000000000', 'slopeRate2': '1000000000000000000'}]} on the lendingPoolCore contract.
1


Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cx75b4cd08dd95e1e6a8ed948653ccb413c5d0573f', 'optimalUtilizationRate': '600000000000000000', 'baseBorrowRate': '000000000000000000', 'slopeRate1': '80000000000000000', 'slopeRate2': '2000000000000000000'}]} on the lendingPoolCore contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


In [49]:
settings_oicx = [{'contract': 'oICX', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
                {'contract': 'oICX', 'method': 'setReserve', 'params':{'_address':contracts['sicx']}},
                {'contract': 'oICX', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                {'contract': 'oICX', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}}]
for sett in settings_oicx:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx3755cee5c580d18c3c46b1816465e765a24d1b85'} on the oICX contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setReserve, with parameters {'_address': 'cx75b4cd08dd95e1e6a8ed948653ccb413c5d0573f'} on the oICX contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setLendingPoolDataProvider, with parameters {'_address': 'cx52fd89a5beeb3a10747a4fd8981b7b866b9bb289'} on the oICX contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setLendingPool, with parameters {'_address': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b'} on the oICX contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


In [51]:
settings_oiusdc = [ {'contract': 'oIUSDC', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
#                     {'contract': 'oIUSDC', 'method': 'setReserve', 'params':{'_address':contracts['iusdc']}},
                    {'contract': 'oIUSDC', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                    {'contract': 'oIUSDC', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}}]
for sett in settings_oiusdc:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx3755cee5c580d18c3c46b1816465e765a24d1b85'} on the oIUSDC contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setLendingPoolDataProvider, with parameters {'_address': 'cx52fd89a5beeb3a10747a4fd8981b7b866b9bb289'} on the oIUSDC contract.
1


Calling setLendingPool, with parameters {'_address': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b'} on the oIUSDC contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


In [52]:
settings_liquidationManager =  [{'contract': 'liquidationManager', 'method': 'setLendingPoolDataProvider', 'params': {'_address':contracts['lendingPoolDataProvider']}},
                                {'contract': 'liquidationManager', 'method': 'setLendingPoolCore', 'params': {'_address':contracts['lendingPoolCore']}},
                                {'contract': 'liquidationManager', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
                                {'contract': 'liquidationManager', 'method': 'setFeeProvider','params': {'_address':contracts['feeProvider']}}
                                ]
for sett in settings_liquidationManager:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolDataProvider, with parameters {'_address': 'cx52fd89a5beeb3a10747a4fd8981b7b866b9bb289'} on the liquidationManager contract.
1


Calling setLendingPoolCore, with parameters {'_address': 'cx3755cee5c580d18c3c46b1816465e765a24d1b85'} on the liquidationManager contract.
1


Calling setPriceOracle, with parameters {'_address': 'cxd902d339f59a0c7344a8d9b92659d8bcaa9c70f9'} on the liquidationManager contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setFeeProvider, with parameters {'_address': 'cx09be6d8080af4be3e03a48a6c1b54955e1feefae'} on the liquidationManager contract.
1


In [53]:
settings_rewards =[ {'contract': 'rewards', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
                    {'contract': 'rewards', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
                    {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
                    {'contract': 'rewards', 'method': 'setOmm','params': {'_address':contracts['ommToken']}},
#                     {'contract': 'rewards', 'method': 'setLpToken','params': {'_address':}},
                    {'contract': 'rewards', 'method': 'setReceipients','params': {'_address':}},
                    {'contract': 'rewards', 'method': 'setSnapsot','params': {'_address':contracts['snapshot']}},
                    {'contract': 'rewards', 'method': 'setStartTimestamp','params': {'_timestamp':}},
                    {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}}]
for sett in settings_rewards:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

SyntaxError: invalid syntax (<ipython-input-53-c739acbb8d86>, line 6)

In [55]:
settings_delegation =[{'contract': 'delegation', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
                     {'contract':  'delegation', 'method': 'setOmmToken','params': {'_address':contracts['ommToken']}}]
for sett in settings_delegation:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cx3755cee5c580d18c3c46b1816465e765a24d1b85'} on the delegation contract.
1


Calling setOmmToken, with parameters {'_address': 'cxe2fe821536890b983bc16bdc6217f740c7e40f8b'} on the delegation contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


In [54]:
settings_ommToken =[{'contract': 'ommToken', 'method': 'setAdmin','params': {'_admin':contracts['rewards']}},
                    {'contract':  'ommToken', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
                    {'contract':  'ommToken', 'method': 'setRewards','params': {'_address':contracts['rewards']}},
                    {'contract':  'ommToken', 'method': 'setMinimumStake','params': {'_min':100 * 10**18}},
                    {'contract':  'ommToken', 'method': 'setUnstakingPeriod','params': {'_time': 120}}]
for sett in settings_ommToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))



Calling setAdmin, with parameters {'_admin': 'cx4f194dfd4db4bdc658be46985a9583a8808cf38e'} on the ommToken contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xe797a92cbf17580ce788895392fafc6159bbed086a4bde0c391e5c60c64d64b6', 'blockHeight': 5021151, 'blockHash': '0x0df1612cfea792e67d09615ef6daf2eddda62cb803ec3761e6bacbdd1b7ab7d1', 'txIndex': 1, 'to': 'cxe2fe821536890b983bc16bdc6217f740c7e40f8b', 'stepUsed': 148920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 148920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x25a42fb7fa8f5249ec5721074444b875631282e74253220cfb3309b47ece8c92', 'blockHeight': 5021153, 'blockHash': '0xb1d2d813169159859bf08f9b03fe2200231ce219e83113cf75c2a61bb5cd6295', 'txIndex': 1, 'to': 'cxe2fe821536890b983bc16bdc6217f740c7e40f8b', 'stepUsed': 150320, 'stepPrice': 10000000000, 'cumulativeStepUsed': 150320, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x9b3fabea77573f53fc0c8b8f7515f82e6479ff2309ca74fc8bc80f91bd7a9c69', 'blockHeight': 5021155, 'blockHash': '0x01cad0bb40acc6f22341b93a07fb86e9a10cd99fd971492b19fdb51b4f26c7ba', 'txIndex': 1, 'to': 'cxe2fe821536890b983bc16bdc6217f740c7e40f8b', 'stepUsed': 141480, 'stepPrice': 10000000000, 'cumulativeStepUsed': 141480, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x8e45b7193cb7af926f91b1b835b84ce1aad8892d710e3fa5274e948925c3df1a', 'blockHeight': 5021157, 'blockHash': '0x5a33f911867b0526757da1fe8cfc53c2dd27d0a04b454a15afa6a71c52e15385', 'txIndex': 1, 'to': 'cxe2fe821536890b983bc16bdc6217f740c7e40f8b', 'stepUsed': 137680, 'stepPrice': 10000000000, 'cumulativeStepUsed': 137680, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [120]:
settings_lendingPoolCore =[ {'contract': 'lendingPoolCore', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
                            {'contract': 'lendingPoolCore', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
                            {'contract': 'lendingPoolCore', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
                            {'contract': 'lendingPoolCore', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
                            {'contract': 'lendingPoolCore', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
                            {'contract': 'lendingPoolCore', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
                            {'contract': 'lendingPoolCore', 'method': 'setStaking','params': {'_address':contracts['staking']}},
                            {'contract': 'lendingPoolCore', 'method': 'set_id','params': {'_value':'1'}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['sample_token'],'_sym':"USDb"}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"IUSDC"}}]
for sett in settings_lendingPoolCore:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPool, with parameters {'_address': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85'} on the lendingPoolCore contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setDaoFund, with parameters {'_address': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075'} on the lendingPoolCore contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setLendingPoolCore, with parameters {'_address': 'cx2d844c9

In [ ]:
settings_snapshot =[{'contract':  'snapshot', 'method': 'setAdmin','params': {'_address':admin}},
                    {'contract':  'snapshot', 'method': 'setStartTimestamp','params': {'_timestamp':}}]
for sett in settings_snapshot:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [121]:
settings_governance =[{'contract': 'governance', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
                      {'contract': 'governance', 'method': 'setRewards','params': {'_address':contracts['rewards']}}]
for sett in settings_governance:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setSnapshot, with parameters {'_address': 'cx584f740fedab472057bd8ee6e32d97b9b0a05b9d'} on the governance contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setRewards, with parameters {'_address': 'cx3e9b334a82529797818ca824cc58c51b52a26087'} on the governance contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


In [56]:
pprint(contracts)

{'addressProvider': 'cx827be16fe4efbf2516ef80795301579c823288a6',
 'daoFund': 'cx97d4f10f4d793c35886b31cc8c70a872ea3b1448',
 'delegation': 'cx7eb823b737f355c08f1911009bb834ffb6773253',
 'feeProvider': 'cx09be6d8080af4be3e03a48a6c1b54955e1feefae',
 'governance': 'cxc3403b1adc439baa0e3553abb25cc6f290432d26',
 'lendingPool': 'cx9ebdfb5b354db434de96b476c49a8fd455d96b1b',
 'lendingPoolCore': 'cx3755cee5c580d18c3c46b1816465e765a24d1b85',
 'lendingPoolDataProvider': 'cx52fd89a5beeb3a10747a4fd8981b7b866b9bb289',
 'liquidationManager': 'cx0242d3eb23c86a8bc32bf13f5681afc75371bec9',
 'oICX': 'cx2bb91424c10a6ded8b5da01994c33b2b3cc99090',
 'oIUSDC': 'cxb75c665e8f0e257b9cbacddc6c7db713362cea99',
 'oToken': 'cxcee03a41d9b57fdf08359a9d40029796a47f100e',
 'ommToken': 'cxe2fe821536890b983bc16bdc6217f740c7e40f8b',
 'priceOracle': 'cxd902d339f59a0c7344a8d9b92659d8bcaa9c70f9',
 'rewards': 'cx4f194dfd4db4bdc658be46985a9583a8808cf38e',
 'sample_token': 'cx195a1294e7836f3f865147e4752a7c7852362c43',
 'sicx': '

In [11]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
            {'contract': 'lendingPool', 'method': 'setLiquidationManagerAddress','params': {'_address':contracts['liquidationManager']}},
            {'contract': 'lendingPool', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']}},
            {'contract': 'lendingPool', 'method': 'setOICXAddress', 'params':{'_address': contracts['oICX']}},
            {'contract': 'lendingPool', 'method': '{'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}}setRewardAddress','params': {'_address':contracts['rewards']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sicx'],'_sym':"ICX"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolAddress', 'params':{'_address': contracts['lendingPool']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationAddress', 'params':{'_address': contracts['liquidationManager']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':5*10**17}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']}},
            {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
            {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
            {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token'],"optimalUtilizationRate":f"8{'0'*17}","baseBorrowRate":f"2{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"1{'0'*18}"} ]}},
            {'contract': 'lendingPool', 'method': 'setSICXAddress', 'params':{'_address': contracts['sicx']}},  
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sicx'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"0{'0'*17}","slopeRate1":f"8{'0'*16}","slopeRate2":f"2{'0'*18}"} ]}},
            {'contract': 'lendingPoolCore', 'method': 'setSnapshot','params': {'_val':contracts['snapshot']}},
            {'contract': 'oICX', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oICX', 'method': 'setReserveAddress', 'params':{'_address':contracts['sicx']}},
            {'contract': 'oICX', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oICX', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'liquidationManager', 'method': 'setDataProviderAddress', 'params': {'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'liquidationManager', 'method': 'setCoreAddress', 'params': {'_address':contracts['lendingPoolCore']}},
            {'contract': 'liquidationManager', 'method': 'setOracleAddress','params': {'_address':contracts['priceOracle']}},
            {'contract': 'liquidationManager', 'method': 'setFeeProviderAddress','params': {'_address':contracts['feeProvider']}},
            {'contract': 'liquidationManager', 'method': 'setFeeProviderAddress','params': {'_address':contracts['feeProvider']}},
            {'contract': 'rewards', 'method': 'setLendingPool','params': {'_val':contracts['lendingPool']}},
            {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_val':contracts['lendingPoolCore']}},
            {'contract': 'rewards', 'method': 'setSnapsot','params': {'_val':contracts['snapshot']}},
            {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}}]

In [23]:
settings=[  {'contract': 'rewards', 'method': 'setDistPercentage','params': {'_deposit':25*10**16,'_borrow':25*10**16,'_ommUSDb':10**17,''}},
            {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_val':contracts['lendingPoolCore']}},
            {'contract': 'rewards', 'method': 'setSnapshot','params': {'_val':contracts['snapshot']}},
            {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}}
            ]

## Rewards settings

In [ ]:
settings=[ {'contract': 'rewards', 'method': 'setLendingPool','params': {'_val':contracts['lendingPool']}},
            {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_val':contracts['lendingPoolCore']}},
            {'contract': 'rewards', 'method': 'setSnapshot','params': {'_val':contracts['snapshot']}},
            {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}}
            ]

In [46]:
value_hex ="0xa9c052e1b91a35634"
value_int = int(value_hex,0)
value = 925937961595273264
print(value/10**18)
print(value_int,value_int / 10 ** 18)

0.9259379615952733
195709883477838222900 195.70988347783822


In [30]:
local_settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**16}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolAddress', 'params':{'_address': contracts['lendingPool']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationAddress', 'params':{'_address': contracts['liquidationManager']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token'],"optimalUtilizationRate":f"8{'0'*17}","baseBorrowRate":f"2{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"1{'0'*18}"} ]}},
              {'contract': 'lendingPool', 'method': 'setSICXAddress', 'params':{'_address': contracts['sicx']}},
                {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'sICX','_quote':'USD','_rate':5*10**17}},
                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sicx'],'_sym':"sICX"}},
                {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
                {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
                {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sicx'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"0{'0'*17}","slopeRate1":f"8{'0'*16}","slopeRate2":f"2{'0'*18}"} ]}},
                {'contract': 'oICX', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
                {'contract': 'oICX', 'method': 'setReserveAddress', 'params':{'_address':contracts['sicx']}},
                {'contract': 'oICX', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                {'contract': 'oICX', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
                {'contract': 'lendingPool', 'method': 'setLiquidationManagerAddress',
'params': {'_address':contracts['liquidationManager']}},
                {'contract': 'liquidationManager', 'method': 'setDataProviderAddress', 'params': {'_address':contracts['lendingPoolDataProvider']}},
                    {'contract': 'liquidationManager', 'method': 'setCoreAddress', 'params': {'_address':contracts['lendingPoolCore']}},
                {'contract': 'liquidationManager', 'method': 'setOracleAddress',
'params': {'_address':contracts['priceOracle']}},
                {'contract': 'liquidationManager', 'method': 'setFeeProviderAddress',
'params': {'_address':contracts['feeProvider']}},]

In [29]:
for sett in local_settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setOracleAddress, with parameters {'_address': 'cx3e335706c9ef4e86fbe1b7d296f7c02689474b9f'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setLendingPoolAddress, with parameters {'_address': 'cx4ce2b2065370a47cb2e4ef1a6a11658f5de1f825'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling setLiquidationAddress, with parameters {'_address': 'cx8985aa3600fb020fba6f399e338a5e3169521f3a'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending

## Adding USDb reserve in LendingPoolCore



In [122]:
params ={"_reserve": {"reserveAddress":contracts['sample_token'],"oTokenAddress":contracts['oToken'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x43e8e462268ba6cdca262d8cf8929df4384d6471469dec989b342dcc45c6d276',
 'blockHeight': 13206499,
 'blockHash': '0xdefceb5d8052e421d3574a50984d13545bcc7c65bea403162300b6f2446d23c5',
 'txIndex': 1,
 'to': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
 'stepUsed': 277040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 277040,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding ICX reserve

In [123]:
params ={"_reserve": {"reserveAddress":contracts['sicx'],"oTokenAddress":contracts['oICX'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x93328a41c1cc6ccd100f4006be0664c8900e7999e58bd2bff54ca07594447b76',
 'blockHeight': 13206505,
 'blockHash': '0xcef30e84ceeea4814192a4ec21dd3d44de5ff4ef91bdb932488104bea2fafa4a',
 'txIndex': 1,
 'to': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
 'stepUsed': 276800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 276800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding IUSDC reserve

In [124]:
params ={"_reserve": {"reserveAddress":contracts['iusdc'],"oTokenAddress":contracts['oIUSDC'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"6","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x67f2e2434792d8cfd6e51633ababf47f5f32530a1c48da74829fa2b3b835ff1d',
 'blockHeight': 13206513,
 'blockHash': '0x8a25cebd819ad9e3087805148b548e201bd303d2d8dfdf121a0f47da5ecf8165',
 'txIndex': 1,
 'to': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
 'stepUsed': 276600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 276600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Transfering USDb to wallets

In [8]:
private2="b3d4e18d733f05dc21f326ba4fe150514874327c738c4a19ddd357d0e8e2efed"
deployer_wallet2 = KeyWallet.load(bytes.fromhex(private2))
print(deployer_wallet2.get_address())

private3="c3d5e9afe753a07c983bb14e34f73907e40a997793ce2f7d2895cddec27f81c6"
deployer_wallet3 = KeyWallet.load(bytes.fromhex(private3))
print(deployer_wallet3.get_address())

private4="2ffb9ef51826d555189d4804161058036cf87ae8290f1a308a79d045f6cdd428"
deployer_wallet4 = KeyWallet.load(bytes.fromhex(private4))
print(deployer_wallet4.get_address())
wallets=[deployer_wallet2,deployer_wallet3,deployer_wallet4]

hxd29037f1eb8c3f0e8630d797edc7410507c0089a
hx66ee5acace86d9ce23127599b7c9daae70c9d7c1
hx65902dd4139d04765534a4539319be874ad7aa0b


In [9]:
for wallet in wallets:
    params = {"_to": wallet.get_address(), "_value": 1000000 * 10 ** 18} 
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts['sample_token'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(3)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


In [17]:
pprint(contracts)

{'addressProvider': 'cx4f0a21a7273a6e797715aa0934581aec8d0bf2c2',
 'feeProvider': 'cxc2b85e43553f7ad1c4104626891e41e54c4c8214',
 'lendingPool': 'cx4ce2b2065370a47cb2e4ef1a6a11658f5de1f825',
 'lendingPoolCore': 'cx7b517bf6a6599aeeabdea3aac78c37861672ac27',
 'lendingPoolDataProvider': 'cx79758fa04b3bc482bb2b42d790fd71594cfc4661',
 'liquidationManager': 'cx8985aa3600fb020fba6f399e338a5e3169521f3a',
 'oICX': 'cxf37ac77ae234396d7c6ef4d8356fbbdbedb2d43e',
 'oToken': 'cx7161c01f5c77c0c18daf472a08ff4db911ce7616',
 'priceOracle': 'cx3e335706c9ef4e86fbe1b7d296f7c02689474b9f',
 'sample_token': 'cx067c5e14f71da93476c6ca5c727345b665bf3148',
 'sicx': 'cx00cfdf6ac95bbdd5d6e14a2be3db760b6a5a0112'}


In [20]:
params = {'_owner': "hx2176d13280d465b68afbf7f9be758e966f654629"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)


print(int(response,0))

1000000000000000000000000


In [19]:
params = {"_to": "hx2176d13280d465b68afbf7f9be758e966f654629", "_value": 1000000 * 10 ** 18}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0xeba90afe431de1c5bb59f475bb2e9312f603428ef5180f7a645342a95e260137', 'blockHeight': 4675475, 'blockHash': '0x3ef16d6ff6fded92063cfdcbf21357fdd27a56430ced2cfe72dce555a2220a7f', 'txIndex': 1, 'to': 'cx067c5e14f71da93476c6ca5c727345b665bf3148', 'stepUsed': 162000, 'stepPrice': 10000000000, 'cumulativeStepUsed': 162000, 'eventLogs': [{'scoreAddress': 'cx067c5e14f71da93476c6ca5c727345b665bf3148', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'hx2176d13280d465b68afbf7f9be758e966f654629', '0xd3c21bcecceda1000000'], 'data': ['0x4e6f6e65']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [10]:
for wallet in wallets:
    print(wallet.get_address())
    params = {"_to": wallet.get_address(), "_value": 1000000 * 10 ** 18}
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts['sicx'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(3)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(get_tx_result(tx_hash))

hxd29037f1eb8c3f0e8630d797edc7410507c0089a
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x3d295167c28f9d45f60e379a5402dab3828fc0b6b9894a7a5161abeb5b5365b3', 'blockHeight': 4538394, 'blockHash': '0x229bd0a9b8e3cee1e0ba4616d71a5cdd14046a9e8a4c09f8a0bf9ccd338d9267', 'txIndex': 1, 'to': 'cx00cfdf6ac95bbdd5d6e14a2be3db760b6a5a0112', 'stepUsed': 162080, 'stepPrice': 10000000000, 'cumulativeStepUsed': 162080, 'eventLogs': [{'scoreAddress': 'cx00cfdf6ac95bbdd5d6e14a2be3db760b6a5a0112', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'hxd29037f1eb8c3f0e8630d797edc7410507c0089a', '0xd3c21bcecceda1000000'], 'data': ['0x4e6f6e65']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00 \x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Transfering icx
 (the below cells are optional,if you want to tranfer icx to any of the wallet than only you need to run the cells)


In [21]:
password = "test1_Account"
icx_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
icx_wallet.get_address()
icon_service.get_balance(icx_wallet.get_address())

702846553760312000000000000

# run the below cell only when you want to transfer ICX to a wallet
change the wallet address and the amount of icx to be transferred to that wallet

In [22]:
# hxd29037f1eb8c3f0e8630d797edc7410507c0089a
# hx66ee5acace86d9ce23127599b7c9daae70c9d7c1
# hx65902dd4139d04765534a4539319be874ad7aa0b
wallet = "hx2176d13280d465b68afbf7f9be758e966f654629" 
amount = 100000*10**18                               #change the amount of icx to transfer here
transaction = TransactionBuilder()\
    .from_(icx_wallet.get_address())\
    .to(wallet)\
    .nid(3) \
    .nonce(100) \
    .value(amount)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(transaction, icx_wallet,step_limit)


# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0xcd90762341303fc505b6c6ca500e7a907961b7cc589cffbc00a185950d36f7ee',
 'blockHeight': 4675502,
 'blockHash': '0xde865ace8b60dba9de40c7859747946f9d456490655d7a5ea1ab21104fff0262',
 'txIndex': 1,
 'to': 'hx2176d13280d465b68afbf7f9be758e966f654629',
 'stepUsed': 100000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 100000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [23]:
contract_ = 'lendingPool'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

update lendingPool cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x4a4510986957f5208462579a2c6d18ceef715f93b9a33a2f786ca3d500357a97',
 'blockHeight': 13370060,
 'blockHash': '0x9c25ed55271f789ec49d2d2f1a7d2d1eb9a7266eed839b77f65b57cbcdd3ccc1',
 'txIndex': 1,
 'to': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'scoreAddress': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'stepUsed': 1738976800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1738976800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [25]:
print(contracts) cx2d844c9613b5f864f5dace7fed82997c4c62ef01

{'addressProvider': 'cx797bc63e7b27758816910cb35f0fa1f546b57c33', 'daoFund': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075', 'delegation': 'cx24e32367425660a67c91d6e3c1077914df4e0286', 'feeProvider': 'cxd55f27870c9b219272ea4fc31d5d7ec8dc78a97e', 'governance': 'cx672132f49f0ef02969cc6b223613206e46ff34b1', 'lendingPool': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85', 'lendingPoolCore': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01', 'lendingPoolDataProvider': 'cx88ae036091243916d87cbc3b11f39e317dcd46ad', 'liquidationManager': 'cx21b6ee09cd144b2a778bdcf69a46ef2cf898ff1c', 'ommToken': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348', 'oToken': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058', 'priceOracle': 'cxe76fd817fae3b485ae6e4fea2cf88d050d379b63', 'rewards': 'cx3e9b334a82529797818ca824cc58c51b52a26087', 'sample_token': 'cxc88e39248bf428d3ab0b9c9fbafef9ff423c3a75', 'snapshot': 'cx584f740fedab472057bd8ee6e32d97b9b0a05b9d', 'worker_token': 'cxd4e8a5dd852439ccdec73a2b4cd973148f09775d', 'oUSDb': 'cx667

In [24]:
print(deployer_wallet.get_address())
params = {"_amount":   10**18 }
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool'])\
    .value(10**18)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("deposit")\
    .params(params)\
    .build()

signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

hx91bf040426f226b3bfcd2f0b5967bbb0320525ce


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xe152b5a402138bd76f9b1cf89ba5f8689eb00dff237a11569ede86d8e28c7608',
 'blockHeight': 13370064,
 'blockHash': '0x37ea19644f988ac6aa9b0d95171f485a2e41224abd20651c5b7813ee54cd4dba',
 'txIndex': 1,
 'to': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'stepUsed': 947180,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 947180,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [70]:
params = {"_amount": 10**18 ,"_reserve":contracts['sicx'] }
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("borrow")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x76ab25fd5ebbcc4cf853a7e9dd6f7c146c0fe582da07e0de71b8a497095d5046',
 'blockHeight': 12742104,
 'blockHash': '0x9e8a2916a7509d73eb45f1fce0a0d07d67053f2ef8aa11fe7e67617043624774',
 'txIndex': 1,
 'to': 'cxec16da74a4eb849cd9bef1f02f1252ce7a88b788',
 'stepUsed': 2306180,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2306180,
 'eventLogs': [{'scoreAddress': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
   'indexed': ['Distribution(str,Address,int)',
    'worker',
    'hxd64a458c9714775e695f63b2d86f66a22c23f85a',
    '0x2a5a058fc295ed000000'],
   'data': []},
  {'scoreAddress': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
   'indexed': ['Distribution(str,Address,int)',
    'daoFund',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    '0x152d02c7e14af6800000'],
   'data': []},
  {'scoreAddress': 'cxd100e81ac0839b6130bf4c6806489a9af6274db6',
   'indexed': ['Distribution(str,Address,int)',
    'ommICX',
    'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
    '0x152d02c7e14af6

In [13]:
contracts['sicx']

'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281'